### STAT 207 Lab 9: Odds Ratios and Logistic Regression

Due: Wednesday, November 6, 23:59:59

### Jaewon Kim    jaewonk3

This lab consists of 3 main parts. You may add as many cells as you need in order to do the exercises. To add cells use the "Insert" tab from the menu bar above and select "insert cell above" or "insert cell below". 

Please use Markdown cells above or below your code cells to explain your results. Make sure graders know that you understand what your code is doing.

**Hint: all of the problems build on the class notes, so the notes are the first place to go for related examples and discussion.**


### Part 1: Association in a 2 x 2 table  (8 pts)



Males between the ages of 11 and 30 underwent arthroscopic surgery for knee injuries, either twisted knee or direct blow injuries. Their recovery from surgery was classified as excellent (E) or poor to good (GFP). The data are tabulated in the data frame created below.

In [1]:
import pandas as pd

In [2]:
# load data from knee surgury study into 2x2 table
tabl = pd.DataFrame({'E':[147, 49], 'GFP':[77+28, 14]}, index=['Twisted', 'Direct'])
tabl

,E,GFP
Twisted,147,105
Direct,49,14


**a) Calculate the odds of an excellent recovery versus fair to good recovery for the two types of injury in this study and display the results. Label them in the output so we know which is which. (2pts)**

In [3]:
odds_t = tabl.iloc[0,0]/tabl.iloc[0,1]
odds_d = tabl.iloc[1,0]/tabl.iloc[1,1]
print("Odds of twisted-excellent vs fair to good:", round(odds_t, 2))
print("Odds of direct-excellent vs fair to good:", round(odds_d, 2))

Odds of twisted-excellent vs fair to good: 1.4
Odds of direct-excellent vs fair to good: 3.5


**b) Calculate the proportion of excellent recovery for the two types of injury. Label them in the output so we know which is which. (2pts)**

In [4]:
prop_et = tabl.iloc[0,0]/(tabl.iloc[0,0] + tabl.iloc[0,1])
prop_ed = tabl.iloc[1,0]/(tabl.iloc[1,0] + tabl.iloc[1,1])
print("Proportion of a twisted type:", round(prop_et, 2))
print("Proportion of a direct type:", round(prop_ed, 2))

Proportion of a twisted type: 0.58
Proportion of a direct type: 0.78


**c) Calculate the odds ratio of excellent recovery for twisted knees versus direct blow injuries. Which type of injury appears to have a better recovery outcome? (1pt)**

In [5]:
odds_et = prop_et/(1 - prop_et)
odds_ed = prop_ed/(1 - prop_ed)
odds_ratio = odds_et/odds_ed
print("Odds ratio:", round(odds_ratio, 2))

Odds ratio: 0.4


Direct blow type appears to have a better recovery outcome.

**d) Calculate a 90% confidence interval for the log-odds-ratio. Does the confidence interval include zero? (3pts)**

In [6]:
import numpy as np
from scipy.stats import norm
lgor = np.log(odds_ratio)
lgor_se = np.sqrt(1/147 + 1/49 + 1/105 + 1/14)
zq = norm.ppf((1 + 0.9)/2)
lc = lgor - zq*lgor_se
hc = lgor + zq*lgor_se
print("Log-odds-ratio:"+str([lc, hc]))

Log-odds-ratio:[-1.457253220797893, -0.37532824295041733]


No it does not include 0 between the interval.

### Part 2: Lead exposure versus gender in the lead study. (8 pts)

**This part uses the lead exposure data previously used in labs 06 and 07. You should have the data in your files as "lead.csv".**

**a) Read the data from 'lead.csv' into a pandas data frame. Create a cross-classification table for the variables 'sex' and 'group'. Save and display the resulting table. (3 pts)**

In [7]:
df = pd.read_csv('lead.csv')
table = pd.crosstab(index = df['sex'], columns = df['group'])
table

group,lead < 40,lead >= 40
sex,,
F,32,16
M,46,30


**b) Calculate the log-odds-ratio for low exposure group for Female versus Male for the 2 x 2 table. Also calculate the standard error of the log-odds-ratio. Display and label both of them. (2 pts)**

In [8]:
orf = table.iloc[0,0]/table.iloc[1,0]
orm = table.iloc[0,1]/table.iloc[1,1]
odds_ratio = orf/orm
lgor = np.log(odds_ratio)
lgor_se = np.sqrt(1/table.iloc[0,0] + 1/table.iloc[0,1] + 1/table.iloc[1,0] + 1/table.iloc[1,1])
print("log-odds-ratio:", lgor)
print("standard error:", lgor_se)

log-odds-ratio: 0.2657031657330057
standard error: 0.38577514664389145


**c) Calculate the z value and p-value for testing the null hypothesis that the population log-odds-ratio equals zero. (Recall how to compute a two-sided p-value for a z test statistic). (2 pts)**

In [9]:
z = lgor/lgor_se
p = 2*norm.cdf(-abs(z))
print("z value:", z)
print("p value:", p)

z value: 0.6887513828833456
p value: 0.490979734986559


**d) Based on your results in c), do you reject or accept the null hypothesis of no association ($H_0$ log-odds-ratio = 0, i.e. odds-ratio=1) at a significance level of $\alpha=0.05$? (1 pt)**

At a significance level of $\alpha=0.05$, we accept the null hypothesis.

### Part 3: More analysis of the lead exposure data. (9 pts)

**a) Using the lead data from Part 2, create a new 0/1 indicator variable in the data frame to encode the low and high exposure groups. Use 1 for the low exposure group and 0 for the high exposure group. Display the first 5 rows of the modified data with the new column that you created. (See logistic regression notes for an example of the pandas 'map' function, which might be helpful in this context). (3 pts)**

In [10]:
df['g'] = df['group'].map({'lead < 40':1, 'lead >= 40':0})
df.head()

,id,age,sex,status,verbiq,perfiq,fulliq,iqtype,totyrs,hyperact,tapping,group,g
0,101,11.083333,M,77,61,85,70,WISC,11,NaN,72.0,lead < 40,1
1,102,9.416667,M,77,82,90,85,WISC,6,0.0,61.0,lead < 40,1
2,103,11.083333,M,30,70,107,86,WISC,5,NaN,49.0,lead < 40,1
3,104,6.916667,M,77,72,85,76,WISC,5,2.0,48.0,lead < 40,1
4,105,11.250000,M,62,72,100,84,WISC,11,NaN,51.0,lead < 40,1


**b) Use the statmodels.formula.api function logit to model the log-odds of being in the low exposure group as a function of 'age' and 'sex'. Display the model summary. Note that the response variable needs to be coded as 0 or 1 depending on the group category. (3 pts)**

In [11]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [12]:
mod = smf.logit(formula = 'g ~ age + sex', data = df).fit()
mod.summary()

Optimization terminated successfully.
         Current function value: 0.648142
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      g   No. Observations:                  124
Model:                          Logit   Df Residuals:                      121
Method:                           MLE   Df Model:                            2
Date:                Sun, 10 Nov 2019   Pseudo R-squ.:                 0.01718
Time:                        23:26:36   Log-Likelihood:                -80.370
converged:                       True   LL-Null:                       -81.774
                                        LLR p-value:                    0.2455
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.1071      0.606     -0.177      0.860      -1.295       1.081
sex[T.M]      -0.1587      0.395     -0.401      0.688      -0.934       0.616
age            0.0835      0.055      1.507      0.132      -0.025       0.192
==============================================================================
"""

**c) In the summary, the logistic regression coefficient for 'age' is the estimated log-odds-ratio for low lead exposure versus high lead exposure associated with a 1 year increase in age, for individuals of the same sex. Based on the results, compute a 95% confidence interval for the odds ratio (not log-odds-raio) corresponding to this coefficient. (2 pts)**

In [13]:
lc = np.exp(-0.025)
hc = np.exp(0.192)
print("95% confidence interval:" +str([lc, hc]))

95% confidence interval:[0.9753099120283326, 1.2116705169649005]


**d) Do either of the two exogenous variables (sex, age) have a significant association with the exposure group, i.e., are either of their coefficients signficantly different from zero at level $\alpha=0.05$? Explain briefly. (1 pt)**

At level of $\alpha=0.05$, none of the coefficients are significantly different from 0.